In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from datetime import datetime
import time
import os
import warnings
from siml.detect_peaks import *
from scipy.signal import savgol_filter
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore")

input_folder = r'C:\Users\A4023862\OneDrive - Astellas Pharma Inc\Forecasting Product\DBM\inputs'
output_folder =  r'C:\Users\A4023862\OneDrive - Astellas Pharma Inc\Forecasting Product\DBM\outputs'

C:\Users\A4023862\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [17]:
######################
# READ IN DATA
######################
input_file = r'cm_202212\last_12_act.csv'
path = os.path.join(input_folder, input_file)
last12_a=pd.read_csv(path)

input_file = r'cm_202212\last_12_fct_loe.csv'
path = os.path.join(input_folder, input_file)
last12_f=pd.read_csv(path)

# Fix column names
last12_a.columns = ['2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01',
       '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01',
       '2022-10-01', '2022-11-01', 'tag', 'cost_objects', 'product',
       'run_type']

last12_f.columns = ['2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01',
       '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01',
       '2022-10-01', '2022-11-01', 'tag', 'cost_objects', 'product',
       'run_type', 'yearly_error', 'rmse', 'model_tag', 'last_12_fct_total']

# Subset
last12_a = last12_a[['2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01',
       '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01',
       '2022-10-01', '2022-11-01', 'tag', 'cost_objects', 'product']]

last12_f = last12_f[['2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01',
       '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01',
       '2022-10-01', '2022-11-01', 'tag', 'cost_objects', 'product']]

# Pivot Table
last12_a=last12_a.melt(id_vars=['tag', 'cost_objects', 'product'], 
        var_name="ds", 
        value_name="value_a")

last12_f=last12_f.melt(id_vars=['tag', 'cost_objects', 'product'], 
        var_name="ds", 
        value_name="value_f")

# Create combines df
last12_c=last12_f.merge(last12_a[['tag', 'ds', 'value_a']], on=['tag', 'ds'], how='inner') 

In [18]:
######################
# TIME SERIES METRICS
######################
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

# Create output df
# output = pd.DataFrame(columns=['tag','mape']) # Output of Phase 1
tags_sdf=[]
sum_a=[]
sum_f=[]
mape_sdf=[]

# Get list of tags
tags = last12_c['tag'].unique()

for tag in tags:
    actual=last12_c[last12_c['tag']==tag]['value_a']
    pred=last12_c[last12_c['tag']==tag]['value_f']
    
    tags_sdf.append(tag)
    sum_a.append(sum(actual))
    sum_f.append(sum(pred))
    mape_sdf.append(mape(actual, pred))
    
# Final Df
output= pd.DataFrame({'tag': tags_sdf,'sum_actuals': sum_a,'sum_pred': sum_f,'mape': mape_sdf})


In [19]:
# Yearly Error calculation
output['ye'] = (( output['sum_pred'] - output['sum_actuals'] ) / output['sum_actuals'] ) * 100
output['abs_ye'] = abs(output['ye'])

# Re-add region, product
q=last12_c[['tag', 'cost_objects', 'product']].drop_duplicates()
output = output.merge(q, on='tag', how='left')

output = output[['tag', 'cost_objects', 'product', 'sum_actuals', 'sum_pred', 'mape', 'ye', 'abs_ye']]

# Save new sales data
output_file = 'ts_metrics.csv'
path = os.path.join(output_folder, output_file)
output.to_csv(path)